<a href="https://colab.research.google.com/github/AmadeusZhang/LabALGA/blob/main/AACAP_Es9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook setup**

In [ ]:
!apt install clang

**Download the code**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

**Organize the code and install**

In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

# **Profiling the Karger and the Karger&Stein mincut algorithm**

In [ ]:
%%writefile karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <random>
#include <stack>
#include <vector>


auto& prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
} 


template <typename node_t>
struct Edge { node_t tail, head; };

/* Represents a graph with n vertices as a collection of edges whose vertices are 
 * indexed between 0 and n - 1. */
template <typename node_t>
struct EdgesVectorGraph
{
    node_t n; // number of vertices
    std::vector<Edge<node_t>> edges;
};


/* An Union-Find data structure (https://en.wikipedia.org/wiki/Disjoint-set_data_structure) is a 
 * data structure that stores a partition of a set into disjoint subsets. Tree height is 
 * controlled using union by size. Uses path compression technique. */
template <typename T>
struct UnionFind
{
    struct Subset { 
        T id, size;

        Subset(T _id, T _size) : id(_id), size(_size) 
        {}
    };
    std::vector<Subset> subsets;
    T nb_subsets;

    UnionFind(T n) : nb_subsets{n} { 
        subsets.reserve(n);
        for (T i = 0; i < n; ++i)
            subsets.emplace_back(i, 1);
    }

    auto find(T x) {
        auto root = x;
        while (root != subsets[root].id) 
        { 
            root = subsets[root].id; 
        }
        while (subsets[x].id != root) 
        { 
            auto id = subsets[x].id; 
            subsets[x].id = root; 
            x = id; 
        }
        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x); 
        auto const j = find(y);
        if (i == j) return;
        if (subsets[i].size < subsets[j].size) 
        { 
            subsets[i].id = j; 
            subsets[j].size += subsets[i].size;
        }
        else 
        {
            subsets[j].id = i; 
            subsets[i].size += subsets[j].size;
        }
        --nb_subsets;
    }

    bool connected(T x, T y) { return find(x) == find(y); }
};


/* A data structure representing a cut of a graph. */
template <typename node_t>
struct GraphCut
{
    std::size_t cut_size;
    UnionFind<node_t> uf; // used to identify the two partitions of nodes
    GraphCut(std::size_t _cut_size, UnionFind<node_t> _uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const& other) const { return cut_size < other.cut_size; }

    auto get_partitions() const {
        std::vector<node_t> P, Q;
        P.reserve(uf.subsets[0].size); 
        Q.reserve(std::size(uf.subsets) - std::size(P));
        for (node_t i = 0; i < std::size(uf.subsets); ++i)
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        return std::array{P, Q};
    }
};


/* Karger's contraction algorithm in O(n + mα(n)) using an Union-Find data structure to keep track
 * of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
 * this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
 * cut. The graph isn't per se modifed, only its vector of edges is shuffled. */
template <typename node_t>
GraphCut<node_t> karger_union_find(EdgesVectorGraph<node_t>& graph)
{
    auto& mt = prng_engine();
    UnionFind uf{graph.n};
    auto start = begin(graph.edges);
    for (int m = size(graph.edges) - 1; uf.nb_subsets != 2; ++start, --m) {
        std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
        uf.merge(start->tail, start->head);
    }
    return {(std::size_t) std::count_if(start, end(graph.edges), [&](auto e)
        { return !uf.connected(e.tail, e.head); }), std::move(uf)};
}


/* Kargen-Stein's contraction recursive algorithm. Instead of using a straighforward recursion, we
 * keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high 
 * probability of obtaining the minimum global cut. */
template <typename node_t>
auto karger_stein_union_find(EdgesVectorGraph<node_t> const& input_graph)
{
    /* A data structure to hold an intermediate contracted graph state. 
     * The Union-Find structure is used to keep track of the merged nodes. */ 
    struct ContractedGraph : EdgesVectorGraph<node_t> { UnionFind<node_t> uf; };

    /* Contracts the given graph until it has 'nb_vertices' vertices. 
     * The graph is not per se modifed, only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph& graph, node_t nb_vertices) {   
        UnionFind uf{graph.uf};
        auto start = begin(graph.edges);
        for (int m = size(graph.edges) - 1; uf.nb_subsets != nb_vertices; ++start, --m) {
            std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
            uf.merge(start->tail, start->head);
        }
        decltype(graph.edges) edges;
        edges.reserve(end(graph.edges) - start);
        std::copy_if(start, end(graph.edges), std::back_inserter(edges),
            [&](auto e){ return !uf.connected(e.tail, e.head); }); // remove self-loops
        return ContractedGraph{nb_vertices, std::move(edges), std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph&& graph){ return GraphCut{std::size(graph.edges), graph.uf}; };
    
    double INV_SQRT_2 = 1.0 / std::sqrt(2);
    GraphCut<node_t> best_minimum_cut{input_graph.n, {{}}};
    std::stack<ContractedGraph, std::vector<ContractedGraph>> graphs;
    graphs.push({input_graph.n, input_graph.edges, {input_graph.n}});

    while (!graphs.empty()) // algorithm's main loop
    {
        auto graph = graphs.top();
        graphs.pop();

        if (graph.n <= 6) {
            best_minimum_cut = std::min(best_minimum_cut, cut(contract(graph, 2)));
        } else {
            node_t t = 1 + std::ceil(graph.n * INV_SQRT_2);
            graphs.push(contract(graph, t));
            graphs.push(contract(graph, t));
        }
    }

    return best_minimum_cut;
}

In [ ]:
%%writefile instance_reader.hpp
#pragma once

#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include "karger.hpp"


template <typename node_t>
EdgesVectorGraph<node_t> read_col_instance(std::string_view file) {
    EdgesVectorGraph<node_t> graph;
    std::ifstream instance;
    instance.open(file.data());
    if (!instance) {
        std::cout << file << std::endl;
        throw std::runtime_error("Such instance doesn't exist. ");
    }
    for (std::string line; std::getline(instance, line);)
    {
        std::istringstream input(line);
        switch(line[0]) {
            case 'p':
                input.ignore(6);
                std::size_t m;
                input >> graph.n >> m;
                graph.edges.reserve(m);
                break;
            case 'e':
                input.ignore(2);
                Edge<node_t> edge;
                input >> edge.tail >> edge.head;
                --edge.tail; --edge.head; // instance files starting vertex is 1
                graph.edges.push_back(std::move(edge));
                break;
            default:
                break;
        }
    }
    return graph;
}

Overwriting instance_reader.hpp


In [ ]:
%%writefile main.cpp
#include <cstdint>
#include <iostream>
#include <functional>
#include <array>
#include <chrono>

#include "karger.hpp"
#include "instance_reader.hpp"


void minimal_example()
{
    EdgesVectorGraph<int> graph{8, {
        {0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}, {4, 5},
        {4, 6}, {4, 7}, {5, 6}, {5, 7}, {6, 7}, {1, 4}, {3, 4}
    }};

    auto cut = karger_union_find(graph); // or karger_stein_union_find(graph)

    std::cout << "Cut's size: " << cut.cut_size << '\n';

    std::cout << "Partitions: ";
    for (auto const& partition : cut.get_partitions()) {
        std::cout << "{ ";
        for (auto u : partition) std::cout << u << ' ';
        std::cout << "} ";
    }
}


int main(int argc, char* argv[])
{
    using node_t = std::uint32_t;
    if (argc != 2) throw std::runtime_error("No input file.");
    auto graph = read_col_instance<node_t>(argv[1]);

    std::cout << "\nInput graph: \"" << argv[1] << "\" (|V| = " << graph.n << ", |E| = "
        << std::size(graph.edges) << ")\n";

    struct MinimumCutAlgorithm {
        std::string name;
        std::function<GraphCut<node_t>(EdgesVectorGraph<node_t>&)> algorithm;
        std::size_t nb_repeat;
        auto operator()(EdgesVectorGraph<node_t>& graph) const { return algorithm(graph); }
    };

    std::array<MinimumCutAlgorithm, 2> algorithms{{
        {"Karger",       karger_union_find<node_t>,       static_cast<std::size_t>(0.5 * graph.n * (graph.n - 1) * std::log(graph.n)) / 6},
        {"Karger-Stein", karger_stein_union_find<node_t>, static_cast<std::size_t>(std::log(graph.n) * std::log(graph.n))}
    }}; 

    for (auto const& algorithm : algorithms)
    {
        std::cout << "\nAlgorithm: \"" << algorithm.name << "\"\n"
                  << "    - Number of repetitions: " << algorithm.nb_repeat << '\n';
        GraphCut<node_t> best_minimum_cut{graph.n, {{}}};
        auto time_start{std::chrono::steady_clock::now()};

        for(std::size_t i = algorithm.nb_repeat; i; --i)
            best_minimum_cut = std::min(best_minimum_cut, algorithm(graph));
        
        auto duration = std::chrono::duration_cast<std::chrono::microseconds>(std::chrono::steady_clock::now() - time_start).count();
        std::cout << "    - Best minimum cut's size found: " << best_minimum_cut.cut_size
                  << "\n    - Duration: " << duration << "us\n";
    }
    
    std::cout << "\n";
}

Overwriting main.cpp


In [ ]:
!clang++ main.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o karger -g

In [ ]:
%%writefile toy.col
p edge 6 9
e 1 2
e 1 3
e 2 3
e 2 4
e 3 4
e 3 5
e 4 5
e 4 6
e 5 6

Overwriting toy.col


In [ ]:
!./karger toy.col


Input graph: "toy.col" (|V| = 6, |E| = 9)

Algorithm: "Karger"
    - Number of repetitions: 4
    - Best minimum cut's size found: 3
    - Duration: 18us

Algorithm: "Karger-Stein"
    - Number of repetitions: 3
    - Best minimum cut's size found: 3
    - Duration: 13us



# References

---
GitHub repo [link](https://github.com/ArthurRouquan/KargerSteinAlgorithm).

